In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import env


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import wrangle2
import wrangle3

In [2]:
train, X_train, y_train, X_validate, y_validate, X_test, y_test=wrangle3.wrangle()

In [3]:
X_train.head()

,Unnamed: 0,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,regionidcity,...,scaled_latitude,scaled_longitude,scaled_bathroomcnt,scaled_taxrate,scaled_bedroomcnt,scaled_lotsizesquarefeet,scaled_age,scaled_acres,scaled_bath_bed_ratio,scaled_calculatedfinishedsquarefeet
21580,21580,11204960,3.0,4.0,2000.0,6037.0,34561695.0,-118082751.0,6999.0,40227.0,...,0.829634,0.731775,0.333333,0.203380,0.500000,0.008825,0.213235,0.008825,0.196429,0.172850
32781,32781,14323862,6.0,6.0,7393.0,6059.0,33601524.0,-117845231.0,24000.0,40009.0,...,0.177142,0.856593,0.833333,0.123723,0.833333,0.031010,0.154412,0.031010,0.285714,0.741252
33161,33161,12205584,2.0,3.0,1778.0,6037.0,33966002.0,-118298627.0,4948.0,12447.0,...,0.424826,0.618331,0.166667,0.124023,0.333333,0.006149,0.588235,0.006149,0.166667,0.149452
45785,45785,12078527,3.0,4.0,2777.0,6037.0,34166013.0,-118218530.0,9989.0,45457.0,...,0.560745,0.660422,0.333333,0.111039,0.500000,0.012727,0.191176,0.012727,0.196429,0.254743
45820,45820,14266520,3.0,4.0,1939.0,6059.0,33706702.0,-117898677.0,6028.0,47568.0,...,0.248617,0.828507,0.333333,0.121312,0.500000,0.007558,0.330882,0.007558,0.196429,0.166421


In [4]:
X_train.shape

(36384, 42)

In [ ]:
X_train.county.value_counts()

In [ ]:
X_train.sample(50)

In [ ]:
df = X_train

In [ ]:
# Viz distributions after outliers removed

for col in df.columns:
    sns.boxplot(df[col])
    plt.title(col)
    plt.show()

## Clustering

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


In [ ]:
train.head()

In [ ]:
X_train.head()

In [ ]:
X_train.dtypes